In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from vonenet.utils import gabor_kernel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GaborFilterBank(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, device, stride=4):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size)
        self.stride = (stride, stride)
        self.padding = (kernel_size // 2, kernel_size//2)

        self.weight = torch.zeros((out_channels, in_channels, kernel_size, kernel_size)).to(device)
        

    def initialize(self, sf, theta, sigx, sigy, phase, seed=None):
        if seed is not None:
            torch.manual_seed(seed)
        
        random_channel = torch.randint(0, self.in_channels, (self.out_channels,))
        for i in range(self.out_channels):
            self.weight[i, random_channel[i]] = gabor_kernel(
                frequency=sf[i], sigma_x=sigx[i], sigma_y=sigy[i],
                theta=theta[i], offset=phase[i], ks=self.kernel_size[0]
            )
        self.weight = nn.Parameter(self.weight, requires_grad=False)

    def forward(self, x):
        # [256, 3, 15, 15] 
        #print("x",x.shape, x.device)
        #print("weight", self.weight.shape, self.weight.device)
        return F.conv2d(x, self.weight, None, stride=self.stride, padding=self.padding)

class NCRFModule(nn.Module):
    def __init__(self, gabor_bank, modulation_strength=0.5, surround_kernel_size=3):
        super(NCRFModule, self).__init__()
        self.gabor_bank = gabor_bank
        self.modulation_strength = modulation_strength
        self.surround_kernel_size = surround_kernel_size

    def forward(self, x):
        #print("here")
        #center_response = self.gabor_bank(x)
        #print("center_response")
        surround_response = F.avg_pool2d(
            x, kernel_size=self.surround_kernel_size, stride=1, padding=self.surround_kernel_size // 2
        ) * self.modulation_strength
        modulated_response = x / (1 + surround_response)
        return modulated_response

class V1Processing(nn.Module):
    def __init__(self, in_channels=3, out_channels=256, kernel_size=9, stride=4, noise_mode='neuronal', noise_level=0.07, noise_scale=0.35, device='cpu'):
        super(V1Processing, self).__init__()
        # self.gabor_bank = GaborFilterBank(in_channels, out_channels, kernel_size)
        self.simple_conv_q0 = GaborFilterBank(in_channels, out_channels, kernel_size, device, stride)
        self.simple_conv_q1 = GaborFilterBank(in_channels, out_channels, kernel_size, device, stride)
        self.ncrf_q0 = NCRFModule(self.simple_conv_q0)
        self.ncrf_q1 = NCRFModule(self.simple_conv_q1)
        
        # Simple and Complex Cells

        # ---- FIX: Initialize Gabor filters with random parameters ----
        sf    = torch.rand(out_channels) * 0.5 + 0.1    # frequencies between 0.1 and 0.6
        theta = torch.rand(out_channels) * np.pi          # orientations between 0 and pi
        sigx  = torch.rand(out_channels) * 2 + 1.0          # sigma between 1 and 3
        sigy  = torch.rand(out_channels) * 2 + 1.0
        phase = torch.rand(out_channels) * 2 * np.pi        # phase between 0 and 2pi
        self.simple_conv_q0.initialize(sf, theta, sigx, sigy, phase)
        self.simple_conv_q0.initialize(sf, theta, sigx, sigy, phase + np.pi/2)

        self.simple = nn.ReLU()
        self.complex = nn.Identity()
        self.gabors = nn.Identity()
        self.noise = nn.ReLU()
        self.output = nn.Identity()
        
        # Noise settings (noise level reduced from 0.1 to 0.01)
        self.noise_mode = noise_mode
        self.noise_level = noise_level
        self.noise_scale = noise_scale
        self.fixed_noise = None
        self.simple_channels = 128 
        self.complex_channels = 128 
        self.k_exc = 25


    def set_noise_mode(self, mode='gaussian', level=0.01):
        self.noise_mode = mode
        self.noise_level = level

    def fix_noise(self, batch_size, shape):
        self.fixed_noise = torch.randn(batch_size, *shape, device=device) * self.noise_level
    
    def noise_f(self, x):
        if self.noise_mode == 'gaussian':
            return x + torch.randn_like(x) * self.noise_level
        elif self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * torch.sqrt(F.relu(x.clone()) + eps)
            else:
                x += torch.distributions.normal.Normal(torch.zeros_like(x), scale=1).rsample() * \
                     torch.sqrt(F.relu(x.clone()) + eps)
            x -= self.noise_level
            x /= self.noise_scale
            return self.noise(x)
        else:
            return x

    def forward(self, x):
        '''
        s_q0 = self.ncrf_q0(self.simple_conv_q0(x))
        s_q1 = self.ncrf_q1(self.simple_conv_q1(x))
        c = self.complex(torch.sqrt(s_q0[:, self.simple_channels:, :, :] ** 2 +
                                    s_q1[:, self.simple_channels:, :, :] ** 2) / np.sqrt(2))
        s = self.simple(s_q0[:, 0:self.simple_channels, :, :])
        response = self.gabors(self.k_exc * torch.cat((s, c), 1))
        
        simple_response = self.noise_f(response)
        output = self.output(simple_response)
        print(output.shape) # torch.Size([32, 3, 14, 14]) 
        return output
        '''
        #print(f"Input shape: {x.shape}")  # Print initial input shape
        #print(self.simple_conv_q0)
        #print(self.ncrf_q0)
        s_q0 = self.simple_conv_q0(x)
        #print(f"Shape after simple_conv_q0 + NCRF: {s_q0.shape}")
    
        s_q1 = self.simple_conv_q1(x)
        #print(f"Shape after simple_conv_q1 + NCRF: {s_q1.shape}")
    
        c = self.complex(torch.sqrt(s_q0[:, self.simple_channels:, :, :] ** 2 +
                                    s_q1[:, self.simple_channels:, :, :] ** 2) / np.sqrt(2))
        #print(f"Shape of complex cell response: {c.shape}")
    
        s = self.simple(s_q0[:, 0:self.simple_channels, :, :])
        #print(f"Shape of simple cell response: {s.shape}")
    
        response = self.gabors(self.k_exc * torch.cat((s, c), 1))
        #print(f"Shape after concatenating simple & complex responses: {response.shape}")
    
        simple_response = self.noise_f(response)
        #print(f"Shape after noise function: {simple_response.shape}")
    
        output = self.output(simple_response)
        #print(f"Final output shape: {output.shape}")

        output = self.ncrf_q0(simple_response)
        #print(f"Final1 output shape: {output.shape}")
    
        return output

In [2]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import alexnet
from collections import OrderedDict

# Import custom Gabor filter bank and processing module from above

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True) #
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True) # inplace=True
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNetBackEnd(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNetBackEnd, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


class ResNetWithGabor(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(ResNetWithGabor, self).__init__()
        gabor = V1Processing(in_channels, out_channels=256, kernel_size=15, stride=4, device='cuda:0').to(device)
        bottleneck = nn.Conv2d(256, 64, kernel_size=1, stride=1, bias=False)
        print(bottleneck)
        nn.init.kaiming_normal_(bottleneck.weight, mode='fan_out', nonlinearity='relu')
        #self.alexnet = alexnet(num_classes=num_classes).to(device)
        
        # Modify first layer to accept 64 Gabor channels instead of 3
        #self.alexnet.features[0] = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        model_back_end = ResNetBackEnd(block=Bottleneck, layers=[3, 4, 6, 3])
        #model_back_end = ResNetBackEnd() 
        print(model_back_end)
        self.model = nn.Sequential(OrderedDict([
            ('ncrf', gabor),
            ('bottleneck',bottleneck),
            ('model', model_back_end)
        ]))

    def forward(self, x):
        x = self.model(x)
        return x

model = ResNetWithGabor().to(device)
print(model)
#model= nn.DataParallel(model).to(device)
criterion = nn.CrossEntropyLoss()
# ---- FIX: Lowered learning rate for Adam (from 0.1 to 1e-3) ----
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def train(model, trainloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            # ---- FIX: Gradient clipping to prevent exploding gradients ----
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

Files already downloaded and verified
Files already downloaded and verified


/home/mrahm326/.local/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
ResNetBackEnd(
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 

In [10]:
torch.save(model.state_dict(), "aniso_res50_cifar10_10_new.pth")
#model.load_state_dict(torch.load("aniso_res50_cifar10_10_new.pth", weights_only=True))

In [5]:
train(model, trainloader, criterion, optimizer, epochs=10)
test(model, testloader)
#torch.save(model.state_dict(), "alexnet_gabor_cifar10.pth")
#print("Model saved successfully!")

Epoch 1/10, Loss: 0.1604
Epoch 2/10, Loss: 0.1090
Epoch 3/10, Loss: 0.0877
Epoch 4/10, Loss: 0.0814
Epoch 5/10, Loss: 0.0675
Epoch 6/10, Loss: 0.0645
Epoch 7/10, Loss: 0.0602
Epoch 8/10, Loss: 0.0513
Epoch 9/10, Loss: 0.0506
Epoch 10/10, Loss: 0.0531
Test Accuracy: 77.91%


In [9]:
train(model, trainloader, criterion, optimizer, epochs=30)
test(model, testloader)
#torch.save(model.state_dict(), "alexnet_gabor_cifar10.pth")
#print("Model saved successfully!")

Epoch 1/30, Loss: 0.0174
Epoch 2/30, Loss: 0.0157
Epoch 3/30, Loss: 0.0191
Epoch 4/30, Loss: 0.0159
Epoch 5/30, Loss: 0.0193
Epoch 6/30, Loss: 0.0131
Epoch 7/30, Loss: 0.0200
Epoch 8/30, Loss: 0.0127
Epoch 9/30, Loss: 0.0176
Epoch 10/30, Loss: 0.0179
Epoch 11/30, Loss: 0.0140
Epoch 12/30, Loss: 0.0183
Epoch 13/30, Loss: 0.0140
Epoch 14/30, Loss: 0.0148
Epoch 15/30, Loss: 0.0167
Epoch 16/30, Loss: 0.0145
Epoch 17/30, Loss: 0.0127
Epoch 18/30, Loss: 0.0140
Epoch 19/30, Loss: 0.0156
Epoch 20/30, Loss: 0.0169


KeyboardInterrupt: 

In [8]:
from utils import *
import torch
import numpy as np
import random

# Set the random seed
seed = 42

# Set seed for random module
random.seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# If using GPU, ensure deterministic behavior (optional)
torch.cuda.manual_seed(seed)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
subset_indices = list(range(1000))  # First 1000 samples
from torch.utils.data import Dataset, DataLoader, Subset
# Create a subset
import random

random_indices = random.sample(range(5000), 1000)
#print(random_indices)

from torch.utils.data import Dataset, DataLoader, Subset
# Create a subset
subset_dataset = Subset(testset, random_indices)

print(len(subset_dataset))
test_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)

testloader = test_loader

loss, acc = evaluate_model_test(model, testloader, device)
pgd_1e3 = pgd_evaluate(model, testloader, device, eps=0.001)
pgd_1e2 = pgd_evaluate(model, testloader, device, eps=0.01)
pgd_1e1 = pgd_evaluate(model, testloader, device, eps=0.1)
cw_1e3 = cw_evaluate(model, testloader, device, c=0.001)
cw_1e2 = cw_evaluate(model, testloader, device, c=0.01)
cw_1e1 = cw_evaluate(model, testloader, device, c=0.1)
print(f"Test Accuracy",acc, f"CW Accuracy - 0.1 : {cw_1e1:.4f} - 0.01 : {cw_1e2:.4f} - 0.001 : {cw_1e3:.4f}")
print(f"Test Accuracy",acc, f"PG Accuracy - 0.1 : {pgd_1e1:.4f} - 0.01 : {pgd_1e2:.4f} - 0.001 : {pgd_1e3:.4f}")

Files already downloaded and verified
1000
Test Accuracy 0.808 CW Accuracy - 0.1 : 0.0000 - 0.01 : 0.5380 - 0.001 : 0.7830
Test Accuracy 0.808 PG Accuracy - 0.1 : 0.4030 - 0.01 : 0.4210 - 0.001 : 0.4520


In [5]:
from utils import *
import torch
import numpy as np
import random

# Set the random seed
seed = 42

# Set seed for random module
random.seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# If using GPU, ensure deterministic behavior (optional)
torch.cuda.manual_seed(seed)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
subset_indices = list(range(1000))  # First 1000 samples
from torch.utils.data import Dataset, DataLoader, Subset
# Create a subset
import random

random_indices = random.sample(range(5000), 1000)
#print(random_indices)

from torch.utils.data import Dataset, DataLoader, Subset
# Create a subset
subset_dataset = Subset(testset, random_indices)

print(len(subset_dataset))
test_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)

testloader = test_loader

loss, acc = evaluate_model_test(model, testloader, device)
pgd_1e3 = pgd_evaluate(model, testloader, device, eps=0.001)
pgd_1e2 = pgd_evaluate(model, testloader, device, eps=0.01)
pgd_1e1 = pgd_evaluate(model, testloader, device, eps=0.1)
cw_1e3 = cw_evaluate(model, testloader, device, c=0.001)
cw_1e2 = cw_evaluate(model, testloader, device, c=0.01)
cw_1e1 = cw_evaluate(model, testloader, device, c=0.1)
print(f"Test Accuracy",acc, f"CW Accuracy - 0.1 : {cw_1e1:.4f} - 0.01 : {cw_1e2:.4f} - 0.001 : {cw_1e3:.4f}")
print(f"Test Accuracy",acc, f"PG Accuracy - 0.1 : {pgd_1e1:.4f} - 0.01 : {pgd_1e2:.4f} - 0.001 : {pgd_1e3:.4f}")

Files already downloaded and verified
1000
Test Accuracy 0.788 CW Accuracy - 0.1 : 0.0000 - 0.01 : 0.4670 - 0.001 : 0.7650
Test Accuracy 0.788 PG Accuracy - 0.1 : 0.3560 - 0.01 : 0.3820 - 0.001 : 0.3890


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
from PIL import Image, ImageFilter
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import vonenet
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import logging
import wandb
from torchvision.datasets import ImageFolder
from utils import load_and_prepare_v1_model, train_model, evaluate_model, initialize_wandb, pgd_attack, load_and_prepare_model
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, L1FMNAttack, L2FMNAttack, L0FMNAttack, L2PGD
import eagerpy as ep

learning_rate = 0.01
momentum = 0.9
weight_decay = 0.0001
batch_size = 1
num_epochs_face = 500
num_epochs_object = 50

class GaussianBlur:
    def __init__(self, sigma):
        self.sigma = sigma

    def __call__(self, img):
        return img.filter(ImageFilter.GaussianBlur(self.sigma))

def get_dataloader(blur_level, batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda img: GaussianBlur(blur_level)(img)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    return trainloader
def adjust_blur_and_train(task_type, model, criterion, optimizer, test_loader, batch_size, device, scheduler):
    if task_type == 'face':
        blur_levels = [8, 4, 2, 1, 0]
        epochs_per_blur = 100
        total_epochs = num_epochs_face
    elif task_type == 'object':
        blur_levels = [8, 4, 2, 1, 0]
        epochs_per_blur = 10
        total_epochs = num_epochs_object
    else:
        raise ValueError("Task type must be 'face' or 'object'")

    current_epoch = 0
    i = 0 
    for blur in blur_levels:
        if current_epoch >= total_epochs:
            break
        epochs_to_train = min(epochs_per_blur, total_epochs - current_epoch)
        train_loader = get_dataloader(blur, batch_size)
        
        if(i==0):
            model, acc = train_model(model, criterion, optimizer, train_loader, test_loader, device, scheduler, num_epochs=10, patience=5)
            #torch.save(model.state_dict(), "model_inference_im100"+str(blur)+".pth")
            #old_params = {name: param.clone() for name, param in model.named_parameters()}
            #fisher_information = compute_fisher_information(model, train_loader, criterion)
        else:
            model, acc = train_model(model, criterion, optimizer, train_loader, test_loader, device, scheduler, num_epochs=10, patience=5)
            #torch.save(model.state_dict(), "model_inference_im100"+str(blur)+".pth")
            #old_params = {name: param.clone() for name, param in model.named_parameters()}
            #fisher_information = compute_fisher_information(model, train_loader, criterion)
        
        i+=1
        current_epoch += epochs_to_train
        
    return model 

In [ ]:
learning_rate = 1e-4
batch_size = 64
optimizer_name = 'sgd'
learning_schedule = 'step'
from torch.optim.lr_scheduler import LambdaLR
project_name = "alex-b2c-cifar10"
initialize_wandb(project_name, {
        'dataset': 'cifar10',
        'architecture': 'alex',
        'epochs': 20,
        'patience': 5
})
device = 'cuda:0'

# Define the learning rate schedule
def lr_lambda(epoch):
    if epoch < 30:
        return 1.0
    elif epoch < 70:
        return 0.1
    elif epoch < 90:
        return 0.01
    elif epoch < 120:
        return 0.001
    else:
        return 0.0005

if learning_schedule == 'step':
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
elif learning_schedule == 'linear':
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1 - epoch / 100)
elif learning_schedule == 'exponential':
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
else:
    scheduler = None

task_type = 'object'
model = AlexNetWithGabor().to(device)
print(model)

model = adjust_blur_and_train(task_type, model, criterion, optimizer, test_loader, batch_size, device, scheduler)
val_loss, acc = evaluate_model(model, test_loader, device)
wandb.finish()

epoch,▁▁
loss,▁
test_accuracy,▁
epoch,1
loss,2.30328
test_accuracy,0.1
test_loss,nan
val_loss,nan


Files already downloaded and verified
Starting training
Epoch 1/10, Loss: 2.3033
Epoch 1/10, Loss: 2.3033
Test Loss: 2.3026, Test Accuracy: 0.1000
Test Loss: 2.3026, Test Accuracy: 0.1000
Epoch 1/10, Validation Loss: 2.3026
Epoch 2/10, Loss: 2.3028
Epoch 2/10, Loss: 2.3028
Test Loss: nan, Test Accuracy: 0.1000
Test Loss: nan, Test Accuracy: 0.1000
Epoch 2/10, Validation Loss: nan
NaN detected in loss
NaN detected in loss


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
from utils import *
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#subset_indices = list(range(1000))  # First 1000 samples
#from torch.utils.data import Dataset, DataLoader, Subset
# Create a subset
#subset_dataset = Subset(testset, subset_indices)
#print(len(subset_dataset))
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

testloader = test_loader

loss, acc = evaluate_model_test(model, testloader, device)
pgd_1e3 = pgd_evaluate(model, testloader, device, eps=0.001)
pgd_1e2 = pgd_evaluate(model, testloader, device, eps=0.01)
pgd_1e1 = pgd_evaluate(model, testloader, device, eps=0.1)
cw_1e3 = cw_evaluate(model, testloader, device, c=0.001)
cw_1e2 = cw_evaluate(model, testloader, device, c=0.01)
cw_1e1 = cw_evaluate(model, testloader, device, c=0.1)
print(f"Test Accuracy",acc, f"CW Accuracy - 0.1 : {cw_1e1:.4f} - 0.01 : {cw_1e2:.4f} - 0.001 : {cw_1e3:.4f}")
print(f"Test Accuracy",acc, f"PG Accuracy - 0.1 : {pgd_1e1:.4f} - 0.01 : {pgd_1e2:.4f} - 0.001 : {pgd_1e3:.4f}")

In [ ]:
# Hyperparameter Optimization with Optuna
import optuna  

def get_dataloaders(batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return trainloader, testloader

def train_and_evaluate(trial):
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    
    trainloader, testloader = get_dataloaders(batch_size)
    model = AlexNetWithGabor().to(device)
    criterion = nn.CrossEntropyLoss()
    
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    model.train()
    for epoch in range(5):  # Limit training epochs for optimization
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
    
    test(model, testloader)
    # Return test accuracy as the optimization metric
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
# To run hyperparameter optimization:

study = optuna.create_study(direction="maximize")
study.optimize(train_and_evaluate, n_trials=20)
print("Best hyperparameters: ", study.best_params)